Copyright (c) 2020, NVIDIA CORPORATION.

Modifications copyright Intel. 

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
import os, time
start = time.time()
very_start = time.time()

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [3]:
import dask
from dask.distributed import Client, wait, LocalCluster
import dask.dataframe as dd

In [4]:

client = Client(n_workers=8, 
                       threads_per_worker=1,
                       memory_limit='200GB',ip='10.1.0.131')
#client = Client(ip='10.2.48.253',memory_limit='100GB')
client

Client Scheduler: tcp://10.1.0.131:42286 Dashboard: http://10.1.0.131:8787/status,Cluster Workers: 8 Cores: 8 Memory: 1.60 TB


# Load Train

In [5]:
%%time
path = '/mnt/DP_disk3/Recsys/nv'
train = dd.read_parquet(f'{path}/train-preproc-fold-*.parquet')#,dtypes=dtypes)
#train = dd.read_parquet('/mnt/DP_disk3/Recsys/train-1.parquet')#,dtypes=dtypes)

CPU times: user 35.3 ms, sys: 4.9 ms, total: 40.2 ms
Wall time: 38.3 ms


In [6]:
train.columns

Index(['tweet_id', 'media', 'links', 'domains', 'tweet_type', 'language',
       'timestamp', 'a_user_id', 'a_follower_count', 'a_following_count',
       'a_is_verified', 'a_account_creation', 'b_user_id', 'b_follower_count',
       'b_following_count', 'b_is_verified', 'b_account_creation',
       'b_follows_a', 'reply', 'retweet', 'retweet_comment', 'like',
       'engage_time', 'len_domains', 'len_hashtags', 'len_links', 'hashtags0',
       'hashtags1', 'fold'],
      dtype='object')

In [7]:
train.dtypes

tweet_id                      uint32
media                          uint8
links                         uint32
domains                       uint32
tweet_type                     uint8
language                       uint8
timestamp             datetime64[ns]
a_user_id                     uint32
a_follower_count              uint32
a_following_count             uint32
a_is_verified                   bool
a_account_creation    datetime64[ns]
b_user_id                     uint32
b_follower_count              uint32
b_following_count             uint32
b_is_verified                   bool
b_account_creation    datetime64[ns]
b_follows_a                     bool
reply                          uint8
retweet                        uint8
retweet_comment                uint8
like                           uint8
engage_time           datetime64[ns]
len_domains                    uint8
len_hashtags                   uint8
len_links                      uint8
hashtags0                     uint32
h

In [8]:
%%time
# DROP UNUSED COLUMNS
#cols_drop = ['links','hashtags0', 'hashtags1', 'fold']
cols_drop = ['links']

train = train.drop(cols_drop,axis=1)

CPU times: user 11 ms, sys: 4.26 ms, total: 15.3 ms
Wall time: 12.2 ms


In [9]:
%%time
train, = dask.persist(train)
print(type(train), train.shape)

<class 'dask.dataframe.core.DataFrame'> (Delayed('int-eb0041e1-7f3d-41cb-9efa-f24991454ea2'), 28)
CPU times: user 52.6 ms, sys: 14.2 ms, total: 66.8 ms
Wall time: 64.7 ms


In [10]:
%%time
train = train.repartition(npartitions=8)
train, = dask.persist(train)
print(type(train), train.shape)

<class 'dask.dataframe.core.DataFrame'> (Delayed('int-5e50928b-a547-42e1-bceb-6cd3ac89d07a'), 28)
CPU times: user 18.8 ms, sys: 1.17 ms, total: 20 ms
Wall time: 17.6 ms


In [11]:
#for i,p in enumerate(train.partitions):
#    print(i,len(p))

In [12]:
label_names = ['reply', 'retweet', 'retweet_comment', 'like']
for col in train.columns:
    if col in label_names:
        train[col] = train[col].astype('float32')
    elif train[col].dtype=='int64':
        train[col] = train[col].astype('int32')
    elif train[col].dtype=='int16':
        train[col] = train[col].astype('int8')

In [13]:
%%time
train = train.reset_index(drop=True)

CPU times: user 5.66 ms, sys: 2.16 ms, total: 7.82 ms
Wall time: 6.26 ms


In [14]:
%%time
train, = dask.persist(train)
print(train.shape)

(Delayed('int-74ca8b69-3c5b-4a93-800b-5f4b211e1527'), 28)
CPU times: user 2.76 ms, sys: 3.74 ms, total: 6.5 ms
Wall time: 5.85 ms


In [15]:
train.head()

,tweet_id,media,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_user_id,b_follower_count,b_following_count,b_is_verified,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,engage_time,len_domains,len_hashtags,len_links,hashtags0,hashtags1,fold
0,0,7,0,1,54,2020-02-09 09:26:50,0,14326,408,False,2018-03-31 21:20:17,2022773,27428,600,False,2018-03-13 13:47:49,True,0.0,0.0,0.0,0.0,1970-01-01 00:00:00,0,0,0,0,0,0
1,10,0,0,1,54,2020-02-09 18:41:35,10,237126,1193,True,2010-11-12 21:39:20,15871988,420,518,False,2011-09-05 16:42:09,False,0.0,0.0,0.0,0.0,1970-01-01 00:00:00,0,0,0,3,0,0
2,20,5,0,1,3,2020-02-09 01:13:28,19,23079,1803,False,2010-07-10 21:39:50,10982964,134,408,False,2016-05-19 02:19:01,False,0.0,0.0,0.0,0.0,1970-01-01 00:00:00,0,0,0,9,0,0
3,30,0,5,2,11,2020-02-07 12:15:20,29,769176,190,False,2009-12-18 14:28:33,15871991,2,29,False,2019-09-10 09:17:08,False,0.0,0.0,0.0,1.0,2020-02-07 12:36:47,0,0,0,0,0,0
4,40,0,0,2,6,2020-02-08 14:14:39,35,73952,13,False,2016-06-21 21:45:25,15871992,2,80,False,2019-12-11 15:38:45,False,0.0,0.0,0.0,1.0,2020-02-09 13:33:47,0,0,0,0,0,0


In [16]:
train.head()['timestamp']



0   2020-02-09 09:26:50
1   2020-02-09 18:41:35
2   2020-02-09 01:13:28
3   2020-02-07 12:15:20
4   2020-02-08 14:14:39
Name: timestamp, dtype: datetime64[ns]

In [17]:
%%time 
# TIME FEATURES
# RAPIDS does this 5x faster than Pandas CPU
# If we didn't need to copy CPU to GPU to CPU, then 1300x faster!
def split_time(df):
    #gf = cudf.from_pandas(df[['timestamp']])
    df['dt_dow']  = df['timestamp'].dt.weekday#.to_array() 
    df['dt_hour'] = df['timestamp'].dt.hour#.to_array()
    df['dt_minute'] = df['timestamp'].dt.minute#.to_array()
    df['dt_second'] = df['timestamp'].dt.second#.to_array()
    return df

train = split_time(train)

CPU times: user 45.4 ms, sys: 8.01 ms, total: 53.4 ms
Wall time: 46.8 ms


In [18]:
train.head()[['engage_time','timestamp']]

,engage_time,timestamp
0,1970-01-01 00:00:00,2020-02-09 09:26:50
1,1970-01-01 00:00:00,2020-02-09 18:41:35
2,1970-01-01 00:00:00,2020-02-09 01:13:28
3,2020-02-07 12:36:47,2020-02-07 12:15:20
4,2020-02-09 13:33:47,2020-02-08 14:14:39


In [19]:
train.timestamp.dtype

dtype('<M8[ns]')

In [20]:
%%time
# ELAPSED TIME
for col in ['engage_time','timestamp']:
    train[col] = train[col].astype('int64')/1e9

CPU times: user 28.8 ms, sys: 3.1 ms, total: 31.9 ms
Wall time: 27.9 ms


In [21]:
%%time
train, = dask.persist(train)
print(type(train), train.shape)

<class 'dask.dataframe.core.DataFrame'> (Delayed('int-37e14440-20f1-4dc1-baed-3455e9a5de80'), 32)
CPU times: user 11.5 ms, sys: 1.86 ms, total: 13.4 ms
Wall time: 11.9 ms


In [22]:
train.head()[['engage_time','timestamp']]

,engage_time,timestamp
0,0.000000e+00,1.581240e+09
1,0.000000e+00,1.581274e+09
2,0.000000e+00,1.581211e+09
3,1.581079e+09,1.581078e+09
4,1.581255e+09,1.581171e+09


In [23]:
def set_nan(ds):
    mask = ds == 0
    ds.loc[mask] = np.nan
    return ds
train['engage_time'] = train['engage_time'].map_partitions(set_nan)

In [24]:
train['elapsed_time'] = train['engage_time'] - train['timestamp']
train['elapsed_time'] = train.elapsed_time.astype('float64')

In [25]:
print(train['elapsed_time'].min().compute(),train['elapsed_time'].max().compute())
print(train['elapsed_time'].mean().compute())

2.0 603956.0
15581.699535245267


In [26]:
train.dtypes

tweet_id                      uint32
media                          uint8
domains                       uint32
tweet_type                     uint8
language                       uint8
timestamp                    float64
a_user_id                     uint32
a_follower_count              uint32
a_following_count             uint32
a_is_verified                   bool
a_account_creation    datetime64[ns]
b_user_id                     uint32
b_follower_count              uint32
b_following_count             uint32
b_is_verified                   bool
b_account_creation    datetime64[ns]
b_follows_a                     bool
reply                        float32
retweet                      float32
retweet_comment              float32
like                         float32
engage_time                  float64
len_domains                    uint8
len_hashtags                   uint8
len_links                      uint8
hashtags0                     uint32
hashtags1                     uint32
f

In [27]:
train.head()

,tweet_id,media,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_user_id,b_follower_count,b_following_count,b_is_verified,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,engage_time,len_domains,len_hashtags,len_links,hashtags0,hashtags1,fold,dt_dow,dt_hour,dt_minute,dt_second,elapsed_time
0,0,7,0,1,54,1.581240e+09,0,14326,408,False,2018-03-31 21:20:17,2022773,27428,600,False,2018-03-13 13:47:49,True,0.0,0.0,0.0,0.0,NaN,0,0,0,0,0,0,6,9,26,50,NaN
1,10,0,0,1,54,1.581274e+09,10,237126,1193,True,2010-11-12 21:39:20,15871988,420,518,False,2011-09-05 16:42:09,False,0.0,0.0,0.0,0.0,NaN,0,0,0,3,0,0,6,18,41,35,NaN
2,20,5,0,1,3,1.581211e+09,19,23079,1803,False,2010-07-10 21:39:50,10982964,134,408,False,2016-05-19 02:19:01,False,0.0,0.0,0.0,0.0,NaN,0,0,0,9,0,0,6,1,13,28,NaN
3,30,0,5,2,11,1.581078e+09,29,769176,190,False,2009-12-18 14:28:33,15871991,2,29,False,2019-09-10 09:17:08,False,0.0,0.0,0.0,1.0,1.581079e+09,0,0,0,0,0,0,4,12,15,20,1287.0
4,40,0,0,2,6,1.581171e+09,35,73952,13,False,2016-06-21 21:45:25,15871992,2,80,False,2019-12-11 15:38:45,False,0.0,0.0,0.0,1.0,1.581255e+09,0,0,0,0,0,0,5,14,14,39,83948.0


# Feature Engineering 

In [28]:
%%time
# TRAIN FIRST 5 DAYS. VALIDATE LAST 2 DAYS
VALID_DOW = [1, 2]# order is [3, 4, 5, 6, 0, 1, 2]
valid = train[train['dt_dow'].isin(VALID_DOW)].reset_index(drop=True)
train = train[~train['dt_dow'].isin(VALID_DOW)].reset_index(drop=True)

CPU times: user 20.3 ms, sys: 6.86 ms, total: 27.2 ms
Wall time: 21.3 ms


In [29]:
%%time
train,valid = dask.persist(train,valid)
print(type(train), train.shape, valid.shape)

<class 'dask.dataframe.core.DataFrame'> (Delayed('int-73e9b328-64ff-4585-9b23-cd31103eb4ea'), 33) (Delayed('int-0ac6fc7c-64b7-4f90-b250-e1a31a2f6ad2'), 33)
CPU times: user 23.2 ms, sys: 3.03 ms, total: 26.3 ms
Wall time: 23 ms


In [30]:
%%time
train = train.set_index('timestamp')
valid = valid.set_index('timestamp')
train,valid = dask.persist(train,valid)
train.head()

CPU times: user 10.4 s, sys: 1.12 s, total: 11.6 s
Wall time: 2min 14s


,tweet_id,media,domains,tweet_type,language,a_user_id,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_user_id,b_follower_count,b_following_count,b_is_verified,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,engage_time,len_domains,len_hashtags,len_links,hashtags0,hashtags1,fold,dt_dow,dt_hour,dt_minute,dt_second,elapsed_time
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.580947e+09,5576485,9,0,2,47,31063,579124,0,False,2016-01-04 22:08:54,26238549,351,582,False,2019-11-12 11:53:25,False,0.0,0.0,0.0,0.0,NaN,0,0,0,0,0,5,3,0,0,0,NaN
1.580947e+09,1104893,0,0,2,11,7686,260668,373,True,2009-12-27 08:20:02,31198872,37,131,False,2012-07-14 08:13:02,False,0.0,0.0,0.0,0.0,NaN,0,0,0,154482,0,3,3,0,0,0,NaN
1.580947e+09,13118504,9,572,2,54,83000,800804,1579,True,2009-02-02 00:18:04,23373173,257,939,False,2013-10-18 18:46:06,False,0.0,0.0,0.0,1.0,1.580948e+09,0,0,0,0,0,4,3,0,0,0,349.0
1.580947e+09,27302728,0,13339,2,3,280982,783906,21107,False,2007-03-16 15:31:51,8181547,2122,2294,False,2012-01-13 23:48:32,False,0.0,0.0,0.0,0.0,NaN,0,0,0,0,0,8,3,0,0,0,NaN
1.580947e+09,3825654,0,607,2,11,6915,389923,196,True,2012-04-08 10:36:45,30471824,48,413,False,2014-12-09 15:41:23,False,0.0,0.0,0.0,0.0,NaN,0,0,0,0,0,4,3,0,0,0,NaN


In [31]:
%%time
train = train.reset_index()
valid = valid.reset_index()
train,valid = dask.persist(train,valid)
train.head()

CPU times: user 366 ms, sys: 42.4 ms, total: 409 ms
Wall time: 3.22 s


,timestamp,tweet_id,media,domains,tweet_type,language,a_user_id,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_user_id,b_follower_count,b_following_count,b_is_verified,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,engage_time,len_domains,len_hashtags,len_links,hashtags0,hashtags1,fold,dt_dow,dt_hour,dt_minute,dt_second,elapsed_time
0,1.580947e+09,5576485,9,0,2,47,31063,579124,0,False,2016-01-04 22:08:54,26238549,351,582,False,2019-11-12 11:53:25,False,0.0,0.0,0.0,0.0,NaN,0,0,0,0,0,5,3,0,0,0,NaN
1,1.580947e+09,1104893,0,0,2,11,7686,260668,373,True,2009-12-27 08:20:02,31198872,37,131,False,2012-07-14 08:13:02,False,0.0,0.0,0.0,0.0,NaN,0,0,0,154482,0,3,3,0,0,0,NaN
2,1.580947e+09,13118504,9,572,2,54,83000,800804,1579,True,2009-02-02 00:18:04,23373173,257,939,False,2013-10-18 18:46:06,False,0.0,0.0,0.0,1.0,1.580948e+09,0,0,0,0,0,4,3,0,0,0,349.0
3,1.580947e+09,27302728,0,13339,2,3,280982,783906,21107,False,2007-03-16 15:31:51,8181547,2122,2294,False,2012-01-13 23:48:32,False,0.0,0.0,0.0,0.0,NaN,0,0,0,0,0,8,3,0,0,0,NaN
4,1.580947e+09,3825654,0,607,2,11,6915,389923,196,True,2012-04-08 10:36:45,30471824,48,413,False,2014-12-09 15:41:23,False,0.0,0.0,0.0,0.0,NaN,0,0,0,0,0,4,3,0,0,0,NaN


In [32]:
#for i,p in enumerate(train.partitions):
#    print(i,len(p))

In [33]:
#for i,p in enumerate(valid.partitions):
#    print(i,len(p))

### Target Encode

In [34]:
class MTE_one_shot:
    
    def __init__(self, folds, smooth, seed=42):
        self.folds = folds
        self.seed = seed
        self.smooth = smooth
        
    def fit_transform(self, train, x_col, y_col, y_mean=None, out_col = None, out_dtype=None):
        
        self.y_col = y_col
        np.random.seed(self.seed)
        
        if 'fold' not in train.columns:
            fsize = len(train)//self.folds
            train['fold'] = 1
            train['fold'] = train['fold'].cumsum()
            train['fold'] = train['fold']//fsize
            train['fold'] = train['fold']%self.folds
        
        if out_col is None:
            tag = x_col if isinstance(x_col,str) else '_'.join(x_col)
            out_col = f'TE_{tag}_{self.y_col}'
        
        if y_mean is None:
            y_mean = train[y_col].mean()#.compute().astype('float32')
        self.mean = y_mean
        
        cols = ['fold',x_col] if isinstance(x_col,str) else ['fold']+x_col
        
        agg_each_fold = train.groupby(cols).agg({y_col:['count','sum']}).reset_index()
        agg_each_fold.columns = cols + ['count_y','sum_y']
        
        agg_all = agg_each_fold.groupby(x_col).agg({'count_y':'sum','sum_y':'sum'}).reset_index()
        cols = [x_col] if isinstance(x_col,str) else x_col
        agg_all.columns = cols + ['count_y_all','sum_y_all']
        
        agg_each_fold = agg_each_fold.merge(agg_all,on=x_col,how='left')
        agg_each_fold['count_y_all'] = agg_each_fold['count_y_all'] - agg_each_fold['count_y']
        agg_each_fold['sum_y_all'] = agg_each_fold['sum_y_all'] - agg_each_fold['sum_y']
        agg_each_fold[out_col] = (agg_each_fold['sum_y_all']+self.smooth*self.mean)/(agg_each_fold['count_y_all']+self.smooth)
        agg_each_fold = agg_each_fold.drop(['count_y_all','count_y','sum_y_all','sum_y'],axis=1)
        
        agg_all[out_col] = (agg_all['sum_y_all']+self.smooth*self.mean)/(agg_all['count_y_all']+self.smooth)
        agg_all = agg_all.drop(['count_y_all','sum_y_all'],axis=1)
        self.agg_all = agg_all
        
        train.columns
        cols = ['fold',x_col] if isinstance(x_col,str) else ['fold']+x_col
        train = train.merge(agg_each_fold,on=cols,how='left')
        del agg_each_fold
        #self.agg_each_fold = agg_each_fold
        #train[out_col] = train.map_partitions(lambda cudf_df: cudf_df[out_col].nans_to_nulls())
        train[out_col] = train[out_col].fillna(self.mean)
        
        if out_dtype is not None:
            train[out_col] = train[out_col].astype(out_dtype)
        return train
    
    def transform(self, test, x_col, out_col = None, out_dtype=None):
        if out_col is None:
            tag = x_col if isinstance(x_col,str) else '_'.join(x_col)
            out_col = f'TE_{tag}_{self.y_col}'
        test = test.merge(self.agg_all,on=x_col,how='left')
        test[out_col] = test[out_col].fillna(self.mean)
        if out_dtype is not None:
            test[out_col] = test[out_col].astype(out_dtype)
        return test
 

TE_media_reply 17.8 seconds<br>
TE_tweet_type_reply 27.1 seconds<br>
TE_language_reply 52.5 seconds<br>
TE_a_user_id_reply 180.0 seconds<br>

In [35]:
%%time
# cuDF TE ENCODING IS SUPER FAST!!
idx = 0; cols = []
start = time.time()
for t in ['reply', 'retweet', 'retweet_comment', 'like']:
    start = time.time()
    for c in ['media', 'tweet_type', 'language', 'a_user_id', 'b_user_id']:
        out_col = f'TE_{c}_{t}'
        encoder = MTE_one_shot(folds=5,smooth=20)
        train = encoder.fit_transform(train, c, t, out_col=out_col, out_dtype='float32')
        valid = encoder.transform(valid, c, out_col=out_col, out_dtype='float32')
        cols.append(out_col)
        train,valid = dask.persist(train,valid)
        del encoder
        #train.head()
        wait(train)
        wait(valid)
        print(out_col,"%.1f seconds"%(time.time()-start))

TE_media_reply 52.7 seconds
TE_tweet_type_reply 100.4 seconds
TE_language_reply 148.3 seconds
TE_a_user_id_reply 301.2 seconds
TE_b_user_id_reply 644.4 seconds
TE_media_retweet 51.5 seconds
TE_tweet_type_retweet 96.5 seconds
TE_language_retweet 142.5 seconds
TE_a_user_id_retweet 307.3 seconds
TE_b_user_id_retweet 634.7 seconds
TE_media_retweet_comment 51.4 seconds
TE_tweet_type_retweet_comment 101.8 seconds
TE_language_retweet_comment 150.1 seconds
TE_a_user_id_retweet_comment 312.7 seconds
TE_b_user_id_retweet_comment 663.4 seconds
TE_media_like 54.4 seconds
TE_tweet_type_like 103.9 seconds
TE_language_like 162.0 seconds
TE_a_user_id_like 336.1 seconds
TE_b_user_id_like 695.5 seconds
CPU times: user 3min 43s, sys: 24 s, total: 4min 7s
Wall time: 43min 58s


In [36]:
train['fold'].value_counts().compute()

5    10755654
1    10753027
8    10748479
9    10734308
2    10731121
7    10728837
6    10706783
0    10699601
3    10692605
4    10686107
Name: fold, dtype: int64

### Multiple Column Target Encode

In [37]:
%%time
# cuDF TE ENCODING IS SUPER FAST!!
idx = 0; cols=[]
c = ['domains','language','b_follows_a','tweet_type','media','a_is_verified']
for t in ['reply', 'retweet', 'retweet_comment', 'like']:
    out_col = f'TE_multi_{t}'
    encoder = MTE_one_shot(folds=5,smooth=20)
    train = encoder.fit_transform(train, c, t, out_col=out_col, out_dtype='float32')
    valid = encoder.transform(valid, c, out_col=out_col, out_dtype='float32')
    cols.append(out_col)
    del encoder

CPU times: user 705 ms, sys: 58.7 ms, total: 764 ms
Wall time: 710 ms


In [38]:
%%time
train,valid = dask.persist(train,valid)
wait(train)
wait(valid)

CPU times: user 34.4 s, sys: 3.98 s, total: 38.4 s
Wall time: 7min 13s


DoneAndNotDoneFutures(done={<Future: finished, type: pandas.DataFrame, key: ('assign-e0b3cdee8fdca41e7953a735096104ed', 6)>, <Future: finished, type: pandas.DataFrame, key: ('assign-e0b3cdee8fdca41e7953a735096104ed', 3)>, <Future: finished, type: pandas.DataFrame, key: ('assign-e0b3cdee8fdca41e7953a735096104ed', 5)>, <Future: finished, type: pandas.DataFrame, key: ('assign-e0b3cdee8fdca41e7953a735096104ed', 4)>, <Future: finished, type: pandas.DataFrame, key: ('assign-e0b3cdee8fdca41e7953a735096104ed', 0)>, <Future: finished, type: pandas.DataFrame, key: ('assign-e0b3cdee8fdca41e7953a735096104ed', 2)>, <Future: finished, type: pandas.DataFrame, key: ('assign-e0b3cdee8fdca41e7953a735096104ed', 7)>, <Future: finished, type: pandas.DataFrame, key: ('assign-e0b3cdee8fdca41e7953a735096104ed', 1)>}, not_done=set())

### Elapsed Time Target Encode

In [39]:
%%time
# cuDF TE ENCODING IS SUPER FAST!!
start = time.time()
idx = 0; cols = []
for c in ['media', 'tweet_type', 'language']:#, 'a_user_id', 'b_user_id']:
    for t in ['elapsed_time']:
        out_col = f'TE_{c}_{t}'
        encoder = MTE_one_shot(folds=5,smooth=20)
        train = encoder.fit_transform(train, c, t, out_col=out_col)
        out_dtype='float32' #if 'user_id' in c else None
        valid = encoder.transform(valid, c, out_col=out_col, out_dtype=out_dtype)
        cols.append(out_col)
        print(out_col,"%.1f seconds"%(time.time()-start))
        #del encoder

TE_media_elapsed_time 0.1 seconds
TE_tweet_type_elapsed_time 0.3 seconds
TE_language_elapsed_time 0.4 seconds
CPU times: user 390 ms, sys: 36.4 ms, total: 426 ms
Wall time: 389 ms


In [40]:
%%time
train,valid = dask.persist(train,valid)
wait(train)
wait(valid)

CPU times: user 20.3 s, sys: 2.25 s, total: 22.6 s
Wall time: 4min 21s


DoneAndNotDoneFutures(done={<Future: finished, type: pandas.DataFrame, key: ('assign-6335ec6dcadb4610c1fcc5f92a1f41bd', 3)>, <Future: finished, type: pandas.DataFrame, key: ('assign-6335ec6dcadb4610c1fcc5f92a1f41bd', 7)>, <Future: finished, type: pandas.DataFrame, key: ('assign-6335ec6dcadb4610c1fcc5f92a1f41bd', 2)>, <Future: finished, type: pandas.DataFrame, key: ('assign-6335ec6dcadb4610c1fcc5f92a1f41bd', 0)>, <Future: finished, type: pandas.DataFrame, key: ('assign-6335ec6dcadb4610c1fcc5f92a1f41bd', 1)>, <Future: finished, type: pandas.DataFrame, key: ('assign-6335ec6dcadb4610c1fcc5f92a1f41bd', 4)>, <Future: finished, type: pandas.DataFrame, key: ('assign-6335ec6dcadb4610c1fcc5f92a1f41bd', 5)>, <Future: finished, type: pandas.DataFrame, key: ('assign-6335ec6dcadb4610c1fcc5f92a1f41bd', 6)>}, not_done=set())

### Count Encode

In [41]:
class FrequencyEncoder:
    
    def __init__(self, seed=42):
        self.seed = seed
        
    def fit_transform(self, train, x_col, c_col=None, out_col = None):
        np.random.seed(self.seed)
        if c_col is None or c_col not in train.columns:
            c_col = 'dummy'
            train[c_col] = 1
            drop = True
        else:
            drop = False
            
        if out_col is None:
            tag = x_col if isinstance(x_col,str) else '_'.join(x_col)
            out_col = f'CE_{tag}_norm'
            
        cols = [x_col] if isinstance(x_col,str) else x_col
        agg_all = train.groupby(cols).agg({c_col:'count'}).reset_index()
        if drop:
            train = train.drop(c_col,axis=1)
        agg_all.columns = cols + [out_col]
        agg_all[out_col] = agg_all[out_col].astype('int32')
        agg_all[out_col] = agg_all[out_col]*1.0/len(train)
        agg_all[out_col] = agg_all[out_col].astype('float32')
    
        train = train.merge(agg_all,on=cols,how='left')
        del agg_all
        #print(train.columns)
        #train[out_col] = train.map_partitions(lambda cudf_df: cudf_df[out_col].nans_to_nulls())
        return train
    
    def transform(self, test, x_col, c_col=None, out_col = None):
        return self.fit_transform(test, x_col, c_col, out_col)
 

In [42]:
class CountEncoder:
    
    def __init__(self, seed=42):
        self.seed = seed
        
    def fit_transform(self, train, test, x_col, out_col = None):
        np.random.seed(self.seed)
        
        common_cols = [i for i in train.columns if i in test.columns and i!=x_col]

        if len(common_cols):
            c_col = common_cols[0]
            drop = False
        else:
            c_col = 'dummy'
            train[c_col] = 1
            test[c_col]=1
            drop = True
            
        if out_col is None:
            tag = x_col if isinstance(x_col,str) else '_'.join(x_col)
            out_col = f'CE_{tag}_norm'
            
        cols = [x_col] if isinstance(x_col,str) else x_col
        agg_all = train.groupby(cols).agg({c_col:'count'}).reset_index()
        agg_all.columns = cols + [out_col]
        
        agg_test = test.groupby(cols).agg({c_col:'count'}).reset_index()
        agg_test.columns = cols + [out_col+'_test']
        agg_all = agg_all.merge(agg_test,on=cols,how='left')
        agg_all[out_col+'_test'] = agg_all[out_col+'_test'].fillna(0)
        agg_all[out_col] = agg_all[out_col] + agg_all[out_col+'_test']
        agg_all = agg_all.drop(out_col+'_test', axis=1)
        del agg_test
            
        if drop:
            train = train.drop(c_col,axis=1)
            test = test.drop(c_col,axis=1)
        train = train.merge(agg_all,on=cols,how='left')
        test = test.merge(agg_all,on=cols,how='left')
        del agg_all
        return train,test

In [43]:
%%time
# cuDF CE ENCODING IS SUPER FAST!!
start = time.time()
idx = 0; cols = []
for c in ['media', 'tweet_type', 'language', 'a_user_id', 'b_user_id']:
    encoder = CountEncoder()
    out_col = f'CE_{c}'
    train,valid = encoder.fit_transform(train, valid, c, out_col=out_col)
    print
    del encoder
    train,valid = dask.persist(train,valid)
    wait(train)
    wait(valid)
    print(out_col,"%.1f seconds"%(time.time()-start))

CE_media 68.8 seconds
CE_tweet_type 152.6 seconds
CE_language 236.8 seconds
CE_a_user_id 366.6 seconds
CE_b_user_id 536.9 seconds
CPU times: user 44.6 s, sys: 4.81 s, total: 49.4 s
Wall time: 8min 56s


In [44]:
%%time
# cuDF CE ENCODING IS SUPER FAST!!
idx = 0; cols = []
start = time.time()
for c in ['media', 'tweet_type', 'language', 'a_user_id', 'b_user_id']:
    encoder = FrequencyEncoder()
    out_col = f'CE_{c}_norm'
    train = encoder.fit_transform(train, c, c_col='tweet_id', out_col=out_col)
    valid = encoder.transform(valid, c, c_col='tweet_id', out_col=out_col)
    cols.append(out_col)
    del encoder
    train,valid = dask.persist(train,valid)
    wait(train)
    wait(valid)
    print(out_col,"%.1f seconds"%(time.time()-start))

CE_media_norm 117.6 seconds
CE_tweet_type_norm 215.3 seconds
CE_language_norm 311.1 seconds
CE_a_user_id_norm 430.2 seconds
CE_b_user_id_norm 588.8 seconds
CPU times: user 48.6 s, sys: 5.3 s, total: 53.9 s
Wall time: 9min 48s


### Difference Encode (Lag Features)

In [45]:
def diff_encode_cudf_v1(train,col,tar,sft=1):
    train[col+'_sft'] = train[col].shift(sft)
    train[tar+'_sft'] = train[tar].shift(sft)
    out_col = f'DE_{col}_{tar}_{sft}'
    train[out_col] = train[tar]-train[tar+'_sft']
    mask = '__MASK__'
    train[mask] = train[col] == train[col+'_sft']
    train = train.drop([col+'_sft',tar+'_sft'],axis=1)
    train[out_col] = train[out_col]*train[mask]
    train = train.drop(mask,axis=1)
    return train

In [46]:
%%time
start = time.time()
# cuDF DE ENCODING IS FAST!!
idx = 0; cols = []; sc = 'timestamp'
for c in ['b_user_id']:
    for t in ['b_follower_count','b_following_count','language']:
        for s in [1,-1]:
            start = time.time()
            train = diff_encode_cudf_v1(train, col=c, tar=t, sft=s)
            valid = diff_encode_cudf_v1(valid, col=c, tar=t, sft=s)
            train,valid = dask.persist(train,valid)
            wait(train)
            wait(valid)
            end = time.time(); idx += 1
            print('DE',c,t,s,'%.1f seconds'%(end-start))

DE b_user_id b_follower_count 1 61.7 seconds
DE b_user_id b_follower_count -1 50.6 seconds
DE b_user_id b_following_count 1 93.4 seconds
DE b_user_id b_following_count -1 49.4 seconds
DE b_user_id language 1 50.1 seconds
DE b_user_id language -1 56.2 seconds
CPU times: user 28.2 s, sys: 3.16 s, total: 31.4 s
Wall time: 6min 1s


### Diff Language

In [47]:
train_lang = train[['a_user_id', 'language', 'tweet_id']].drop_duplicates()
valid_lang = valid[['a_user_id', 'language', 'tweet_id']].drop_duplicates()
train_lang_count = train_lang.groupby(['a_user_id', 'language']).agg({'tweet_id':'count'}).reset_index()
valid_lang_count = valid_lang.groupby(['a_user_id', 'language']).agg({'tweet_id':'count'}).reset_index()
train_lang_count,valid_lang_count = dask.persist(train_lang_count,valid_lang_count)
train_lang_count.head()
del train_lang,valid_lang

In [48]:
%%time
train_lang_count = train_lang_count.merge(valid_lang_count,on=['a_user_id', 'language'],how='left')
train_lang_count['tweet_id_y'] = train_lang_count['tweet_id_y'].fillna(0)
train_lang_count['tweet_id_x'] = train_lang_count['tweet_id_x'] + train_lang_count['tweet_id_y']
train_lang_count = train_lang_count.drop('tweet_id_y',axis=1)
train_lang_count.columns = ['a_user_id', 'top_language', 'language_count']
train_lang_count, = dask.persist(train_lang_count)
train_lang_count.head()

CPU times: user 986 ms, sys: 130 ms, total: 1.12 s
Wall time: 11.4 s


,a_user_id,top_language,language_count
0,0,3,3.0
1,0,4,1.0
2,0,11,21.0
3,0,13,3.0
4,0,38,1.0


In [49]:
%%time

train_lang_count = train_lang_count.sort_values(['a_user_id', 'language_count'])
train_lang_count['a_user_shifted'] = train_lang_count['a_user_id'].shift(1)
train_lang_count = train_lang_count[train_lang_count['a_user_id']!=train_lang_count['a_user_shifted']]
train_lang_count = train_lang_count.drop(['a_user_shifted','language_count'],axis=1)
train_lang_count.columns = ['a_user_id','top_language']
train_lang_count, = dask.persist(train_lang_count)
train_lang_count.head()

AttributeError: 'DataFrame' object has no attribute 'sort_values'

In [50]:
def diff_language(df,df_lang_count):
    df = df.merge(df_lang_count,how='left', left_on='b_user_id', right_on='a_user_id')
    df['nan_language'] = df['top_language'].isnull()
    df['same_language'] = df['language'] == df['top_language']
    df['diff_language'] = df['language'] != df['top_language']
    df['same_language'] = df['same_language']*(1-df['nan_language'])
    df['diff_language'] = df['diff_language']*(1-df['nan_language'])
    df = df.drop('top_language',axis=1)
    return df

In [51]:
#%%time
#train = diff_language(train,train_lang_count)
#valid = diff_language(valid,train_lang_count)
#train,valid = dask.persist(train,valid)
#train.head()

## Rate feature

In [52]:
%%time
# follow rate feature
train['a_ff_rate'] = (train['a_following_count'] / train['a_follower_count']).astype('float32')
train['b_ff_rate'] = (train['b_follower_count']  / train['b_following_count']).astype('float32')
valid['a_ff_rate']  = (valid['a_following_count'] / valid['a_follower_count']).astype('float32')
valid['b_ff_rate']  = (valid['b_follower_count']  / valid['b_following_count']).astype('float32')

CPU times: user 54 ms, sys: 9.19 ms, total: 63.2 ms
Wall time: 54.1 ms


In [53]:
train,valid = dask.persist(train,valid)

In [54]:
wait(train)
wait(valid)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.DataFrame, key: ('assign-b43c236fd85fa273da501a552a3b9fae', 3)>, <Future: finished, type: pandas.DataFrame, key: ('assign-b43c236fd85fa273da501a552a3b9fae', 1)>, <Future: finished, type: pandas.DataFrame, key: ('assign-b43c236fd85fa273da501a552a3b9fae', 7)>, <Future: finished, type: pandas.DataFrame, key: ('assign-b43c236fd85fa273da501a552a3b9fae', 5)>, <Future: finished, type: pandas.DataFrame, key: ('assign-b43c236fd85fa273da501a552a3b9fae', 6)>, <Future: finished, type: pandas.DataFrame, key: ('assign-b43c236fd85fa273da501a552a3b9fae', 2)>, <Future: finished, type: pandas.DataFrame, key: ('assign-b43c236fd85fa273da501a552a3b9fae', 4)>, <Future: finished, type: pandas.DataFrame, key: ('assign-b43c236fd85fa273da501a552a3b9fae', 0)>}, not_done=set())

# Summarize Features

In [55]:
%%time

label_names = ['reply', 'retweet', 'retweet_comment', 'like']
DONT_USE = ['tweet_id','timestamp','a_account_creation','b_account_creation','engage_time',
            'fold','b_user_id','a_user_id', 'dt_dow',
            'a_account_creation', 'b_account_creation', 'elapsed_time',
             'links','domains','hashtags0','hashtags1']
DONT_USE += label_names
features = [c for c in train.columns if c not in DONT_USE]

RMV = [c for c in DONT_USE if c in train.columns and c not in label_names]
RMV

CPU times: user 151 µs, sys: 0 ns, total: 151 µs
Wall time: 157 µs


['tweet_id',
 'timestamp',
 'a_account_creation',
 'b_account_creation',
 'engage_time',
 'fold',
 'b_user_id',
 'a_user_id',
 'dt_dow',
 'a_account_creation',
 'b_account_creation',
 'elapsed_time',
 'domains',
 'hashtags0',
 'hashtags1']

In [56]:
%%time

for col in RMV:
    #print(col, col in train.columns)
    if col in train.columns:
        train = train.drop(col,axis=1)
        train, = dask.persist(train)
        train.head()
        

CPU times: user 6.5 s, sys: 720 ms, total: 7.22 s
Wall time: 1min 13s


In [57]:
train.dtypes 

media                                  uint8
tweet_type                             uint8
language                               uint8
a_follower_count                      uint32
a_following_count                     uint32
a_is_verified                         object
b_follower_count                      uint32
b_following_count                     uint32
b_is_verified                           bool
b_follows_a                           object
reply                                float32
retweet                              float32
retweet_comment                      float32
like                                 float32
len_domains                            uint8
len_hashtags                           uint8
len_links                              uint8
dt_hour                                int64
dt_minute                              int64
dt_second                              int64
TE_media_reply                       float32
TE_tweet_type_reply                  float32
TE_languag

In [58]:
%%time

for col in RMV:
    #print(col, col in valid.columns)
    if col in valid.columns:
        valid = valid.drop(col,axis=1)
        valid, = dask.persist(valid,)
        valid.head()
        

CPU times: user 4.48 s, sys: 414 ms, total: 4.89 s
Wall time: 49.2 s


In [59]:
valid.dtypes

media                                  uint8
tweet_type                             uint8
language                               uint8
a_follower_count                      uint32
a_following_count                     uint32
a_is_verified                         object
b_follower_count                      uint32
b_following_count                     uint32
b_is_verified                           bool
b_follows_a                           object
reply                                float32
retweet                              float32
retweet_comment                      float32
like                                 float32
len_domains                            uint8
len_hashtags                           uint8
len_links                              uint8
dt_hour                                int64
dt_minute                              int64
dt_second                              int64
TE_media_reply                       float32
TE_tweet_type_reply                  float32
TE_languag

# Train Model Validate
We will train on random 10% of first 5 days and validation on last 2 days

In [60]:
%%time

SAMPLE_RATIO = 0.1
SEED = 1

if SAMPLE_RATIO < 1.0:
    print(len(train))
    train = train.sample(frac=SAMPLE_RATIO,random_state=42)
    train, = dask.persist(train)
    train.head()
    print(len(train))

train = train.compute()
Y_train = train[label_names]
train = train.drop(label_names,axis=1)

features = [c for c in train.columns if c not in DONT_USE]
print('Using %i features:'%(len(features)),train.shape[1])
np.asarray(features)

107236522
10723652
Using 61 features: 61
CPU times: user 3min 3s, sys: 1min 2s, total: 4min 5s
Wall time: 1min 34s


array(['media', 'tweet_type', 'language', 'a_follower_count',
       'a_following_count', 'a_is_verified', 'b_follower_count',
       'b_following_count', 'b_is_verified', 'b_follows_a', 'len_domains',
       'len_hashtags', 'len_links', 'dt_hour', 'dt_minute', 'dt_second',
       'TE_media_reply', 'TE_tweet_type_reply', 'TE_language_reply',
       'TE_a_user_id_reply', 'TE_b_user_id_reply', 'TE_media_retweet',
       'TE_tweet_type_retweet', 'TE_language_retweet',
       'TE_a_user_id_retweet', 'TE_b_user_id_retweet',
       'TE_media_retweet_comment', 'TE_tweet_type_retweet_comment',
       'TE_language_retweet_comment', 'TE_a_user_id_retweet_comment',
       'TE_b_user_id_retweet_comment', 'TE_media_like',
       'TE_tweet_type_like', 'TE_language_like', 'TE_a_user_id_like',
       'TE_b_user_id_like', 'TE_multi_reply', 'TE_multi_retweet',
       'TE_multi_retweet_comment', 'TE_multi_like',
       'TE_media_elapsed_time', 'TE_tweet_type_elapsed_time',
       'TE_language_elapsed_tim

In [61]:
train.dtypes

media                                  uint8
tweet_type                             uint8
language                               uint8
a_follower_count                       int64
a_following_count                      int64
a_is_verified                           bool
b_follower_count                       int64
b_following_count                      int64
b_is_verified                           bool
b_follows_a                             bool
len_domains                            uint8
len_hashtags                           uint8
len_links                              uint8
dt_hour                                int64
dt_minute                              int64
dt_second                              int64
TE_media_reply                       float32
TE_tweet_type_reply                  float32
TE_language_reply                    float32
TE_a_user_id_reply                   float32
TE_b_user_id_reply                   float32
TE_media_retweet                     float32
TE_tweet_t

In [62]:
SAMPLE_RATIO = 0.35 # VAL SET NOW SIZE OF TEST SET
SEED = 1
if SAMPLE_RATIO < 1.0:
    print(len(valid))
    valid = valid.sample(frac=SAMPLE_RATIO,random_state=42)
    valid, = dask.persist(valid)
    valid.head()
    print(len(valid))
    
valid = valid.compute()
Y_valid = valid[label_names]
valid = valid.drop(label_names,axis=1)

40838716
14293552


In [63]:
import xgboost as xgb
print('XGB Version',xgb.__version__)

xgb_parms = { 
    'max_depth':8, 
    'learning_rate':0.1, 
    'subsample':0.8,
    'colsample_bytree':0.3, 
    'eval_metric':'logloss',
    'objective':'binary:logistic',
    'nthread':40,
    'tree_method':'hist',
    #'predictor' : 'gpu_predictor'
}


XGB Version 1.2.1


In [64]:
if train.columns.duplicated().sum()>0:
    raise Exception(f'duplicated!: { train.columns[train.columns.duplicated()] }')
print('no dup :) ')
print(f'X_train.shape {train.shape}')
print(f'X_valid.shape {valid.shape}')

no dup :) 
X_train.shape (10723652, 61)
X_valid.shape (14293552, 61)


In [65]:
train.dtypes

media                                  uint8
tweet_type                             uint8
language                               uint8
a_follower_count                       int64
a_following_count                      int64
a_is_verified                           bool
b_follower_count                       int64
b_following_count                      int64
b_is_verified                           bool
b_follows_a                             bool
len_domains                            uint8
len_hashtags                           uint8
len_links                              uint8
dt_hour                                int64
dt_minute                              int64
dt_second                              int64
TE_media_reply                       float32
TE_tweet_type_reply                  float32
TE_language_reply                    float32
TE_a_user_id_reply                   float32
TE_b_user_id_reply                   float32
TE_media_retweet                     float32
TE_tweet_t

In [66]:
%%time

for col in train.columns:
    if train[col].dtype=='bool':
        train[col] = train[col].astype('int8')
        valid[col] = valid[col].astype('int8')

CPU times: user 2.16 s, sys: 147 ms, total: 2.3 s
Wall time: 59.1 ms


In [67]:
train.dtypes

media                                  uint8
tweet_type                             uint8
language                               uint8
a_follower_count                       int64
a_following_count                      int64
a_is_verified                           int8
b_follower_count                       int64
b_following_count                      int64
b_is_verified                           int8
b_follows_a                             int8
len_domains                            uint8
len_hashtags                           uint8
len_links                              uint8
dt_hour                                int64
dt_minute                              int64
dt_second                              int64
TE_media_reply                       float32
TE_tweet_type_reply                  float32
TE_language_reply                    float32
TE_a_user_id_reply                   float32
TE_b_user_id_reply                   float32
TE_media_retweet                     float32
TE_tweet_t

In [68]:
%%time
# TRAIN AND VALIDATE

NROUND = 300
VERBOSE_EVAL = 50
#ESR = 50
    
oof = np.zeros((len(valid),len(label_names)))
preds = []
for i in range(4):

    name = label_names[i]
    print('#'*25);print('###',name);print('#'*25)
       
    start = time.time(); print('Creating DMatrix...')
        
    dtrain = xgb.DMatrix(data=train,label=Y_train.iloc[:, i])
    dvalid = xgb.DMatrix(data=valid,label=Y_valid.iloc[:, i])
    print('Took %.1f seconds'%(time.time()-start))
             
    start = time.time(); print('Training...')
    model = xgb.train(xgb_parms, 
                           dtrain=dtrain,
                           #evals=[(dtrain,'train'),(dvalid,'valid')],
                           num_boost_round=NROUND,
                           #early_stopping_rounds=ESR,
                           verbose_eval=VERBOSE_EVAL) 
    print('Took %.1f seconds'%(time.time()-start))
        
    start = time.time(); print('Predicting...')
    #Y_valid[f'pred_{name}'] = xgb.dask.predict(client,model,valid)
    oof[:, i] += model.predict(dvalid)
    #preds.append(xgb.dask.predict(client,model,valid))
    print('Took %.1f seconds'%(time.time()-start))
        
    del model, dtrain, dvalid

#########################
### reply
#########################
Creating DMatrix...
Took 55.8 seconds
Training...
Took 163.5 seconds
Predicting...
Took 11.0 seconds
#########################
### retweet
#########################
Creating DMatrix...
Took 55.3 seconds
Training...
Took 164.7 seconds
Predicting...
Took 10.6 seconds
#########################
### retweet_comment
#########################
Creating DMatrix...
Took 56.2 seconds
Training...
Took 163.1 seconds
Predicting...
Took 10.7 seconds
#########################
### like
#########################
Creating DMatrix...
Took 52.3 seconds
Training...
Took 155.2 seconds
Predicting...
Took 10.6 seconds
CPU times: user 6h 52min 15s, sys: 42min 23s, total: 7h 34min 38s
Wall time: 15min 13s


In [69]:
yvalid = Y_valid[label_names].values

# Compute Validation Metrics

In [70]:
from sklearn.metrics import precision_recall_curve, auc, log_loss

def compute_prauc(pred, gt):
  prec, recall, thresh = precision_recall_curve(gt, pred)
  prauc = auc(recall, prec)
  return prauc

def calculate_ctr(gt):
  positive = len([x for x in gt if x == 1])
  ctr = positive/float(len(gt))
  return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

# FAST METRIC FROM GIBA
def compute_rce_fast(pred, gt):
    cross_entropy = log_loss(gt, pred)
    yt = np.mean(gt)     
    strawman_cross_entropy = -(yt*np.log(yt) + (1 - yt)*np.log(1 - yt))
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

In [71]:
%%time
txt = ''
for i in range(4):
    prauc = compute_prauc(oof[:,i], yvalid[:, i])
    rce   = compute_rce_fast(oof[:,i], yvalid[:, i])
    txt_ = f"{label_names[i]:20} PRAUC:{prauc:.5f} RCE:{rce:.5f}"
    print(txt_)
    txt += txt_ + '\n'

reply                PRAUC:0.17809 RCE:19.30924
retweet              PRAUC:0.53406 RCE:29.33703
retweet_comment      PRAUC:0.05406 RCE:11.29481
like                 PRAUC:0.78780 RCE:28.17722
CPU times: user 14min 51s, sys: 1min 4s, total: 15min 56s
Wall time: 40.2 s


In [72]:
print('This notebook took %.1f minutes'%((time.time()-very_start)/60.))

This notebook took 106.5 minutes
